In [2]:
# crea graficos interactivos
%matplotlib inline
from astropy.io import fits #para acceder a los archivos FITS
from astropy.table import Table # para crear tablas ordenadas de los datos
import matplotlib.pyplot as plt # para plotear datos
import pandas as pd
import numpy as np
import glob
import math
import os
import random as rnd

# mostrar max columnas necesarias
pd.set_option('display.max_columns', 500) 

In [3]:
df_koi = pd.read_csv("./mast-info.csv", skiprows=53)
df_koi

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443.0,NaN,NaN,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,1.252,0.051,-0.049,3.22210,0.01740,-0.01740,1579.2,4.6,-4.6,29.35,7.70,-2.57,2088.0,NaN,NaN,4500.53,3406.38,-1175.26,453.3,1.0,q1_q17_dr25_tce,5638.0,139.0,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9560,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,0.043,0.423,-0.043,3.11400,0.22900,-0.22900,48.5,5.4,-5.4,0.72,0.24,-0.08,1608.0,NaN,NaN,1585.81,1537.86,-502.22,10.6,1.0,q1_q17_dr25_tce,6119.0,165.0,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9561,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,0,0.681402,2.434000e-06,-2.434000e-06,132.181750,0.002850,-0.002850,0.147,0.309,-0.147,0.86500,0.16200,-0.16200,103.6,14.7,-14.7,1.07,0.36,-0.11,2218.0,NaN,NaN,5713.41,5675.74,-1836.94,12.3,1.0,q1_q17_dr25_tce,6173.0,193.0,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385
9562,10155286,K07988.01,NaN,CANDIDATE,CANDIDATE,0.092,0,0,0,0,333.486169,4.235000e-03,-4.235000e-03,153.615010,0.005070,-0.005070,0.214,0.255,-0.214,3.19900,0.22900,-0.22900,639.1,52.7,-52.7,19.30,0.55,-4.68,557.0,NaN,NaN,22.68,2.07,-10.95,14.0,1.0,q1_q17_dr25_tce,4989.0,39.0,-128.0,2.992,0.030,-0.0

In [4]:
# coge las n filas centrales del dataframe
def middle_rows(df, n):
    if df.shape[0] > n:
        half = n // 2
        n = df.shape[0] // 2
        odd = 0 if df.shape[0] % 2 else 1
        eager = 1 if df.shape[0] % 2 else 0
        df = df.iloc[n-half-odd : n+half+eager]
    return df

In [5]:
def local_or_global_file(row, common_shape, scope):
    kepoi_folder = "./lightkurve/" + row["kepoi_name"]
    kepoi_file_global = kepoi_folder + "/" + row["kepoi_name"] + "_global.csv"
    kepoi_file_local = kepoi_folder + "/" + row["kepoi_name"] + "_local.csv"

    df_used = None
    if scope == 'local' and not isinstance(df_used, pd.DataFrame) and os.path.exists(kepoi_file_local):
        df_local = pd.read_csv(kepoi_file_local)
        df_used = df_local if df_local.shape[0] >= common_shape else None
    
    if scope == 'global' and not isinstance(df_used, pd.DataFrame) and os.path.exists(kepoi_file_global):
        df_global = pd.read_csv(kepoi_file_global)
        df_used = df_global if df_global.shape[0] >= common_shape else None
    
    return df_used


In [34]:
len_df_koi = len(df_koi)

modes = [
    {
        'scope': 'local', 
        'bins': 101,
        'df': None
    }, 
    {
        'scope': 'global', 
        'bins': 1001,
        'df': None
    }
]

for x in range(0, len(modes)):
    df_unified = pd.DataFrame()
    for i in range(0, len_df_koi):
    # for i in range(0, 100):
        row = df_koi.iloc[i]
        
        df_flux = local_or_global_file(row, modes[x]['bins'], modes[x]['scope'])
        if isinstance(df_flux, pd.DataFrame):
            df_flux = middle_rows(df_flux, modes[x]['bins'])
            if df_flux.shape[0] == modes[x]['bins']:
                df_flux = df_flux.reset_index(drop=True).drop(columns="Unnamed: 0").transpose()
                df_flux.columns = ['light_flux_' + str((int(col) + 1)).zfill(4) for col in df_flux.columns]

                # df_flux.plot()
                df_flux = row[0:4].append(df_flux.iloc[0])
                # display(df_flux)
                df_unified = df_unified.append(df_flux, ignore_index=True)

    df_unified.loc[df_unified["koi_disposition"] != "CONFIRMED", "koi_disposition"] = 0
    df_unified.loc[df_unified["koi_disposition"] == "CONFIRMED", "koi_disposition"] = 1
    modes[x]['df'] = df_unified

In [45]:
# eliminamos entradas duplicadas de kepids
x1 = modes[0]['df'].drop_duplicates(subset='kepid', keep="first")
x2 = modes[1]['df'].drop_duplicates(subset='kepid', keep="first")

# kepids que no estén en ambos dataframes
sym_difference = list(set(x1["kepid"].tolist()) ^ set(x2["kepid"].tolist()))

# quitamos de los df aquellas filas cuyo kepid no esté en ambos dfs
x1 = x1[ ~ x1['kepid'].isin(sym_difference)]
x2 = x2[ ~ x2['kepid'].isin(sym_difference)]

modes[0]['df'] = x1
modes[1]['df'] = x2

In [71]:
x1

,kepid,kepler_name,kepoi_name,koi_disposition,light_flux_0001,light_flux_0002,light_flux_0003,light_flux_0004,light_flux_0005,light_flux_0006,light_flux_0007,light_flux_0008,light_flux_0009,light_flux_0010,light_flux_0011,light_flux_0012,light_flux_0013,light_flux_0014,light_flux_0015,light_flux_0016,light_flux_0017,light_flux_0018,light_flux_0019,light_flux_0020,light_flux_0021,light_flux_0022,light_flux_0023,light_flux_0024,light_flux_0025,light_flux_0026,light_flux_0027,light_flux_0028,light_flux_0029,light_flux_0030,light_flux_0031,light_flux_0032,light_flux_0033,light_flux_0034,light_flux_0035,light_flux_0036,light_flux_0037,light_flux_0038,light_flux_0039,light_flux_0040,light_flux_0041,light_flux_0042,light_flux_0043,light_flux_0044,light_flux_0045,light_flux_0046,light_flux_0047,light_flux_0048,light_flux_0049,light_flux_0050,light_flux_0051,light_flux_0052,light_flux_0053,light_flux_0054,light_flux_0055,light_flux_0056,light_flux_0057,light_flux_0058,light_flux_0059,light_flux_0060,light_flux_0061,light_flux_0062,light_flux_0063,light_flux_0064,light_flux_0065,light_flux_0066,light_flux_0067,light_flux_0068,light_flux_0069,light_flux_0070,light_flux_0071,light_flux_0072,light_flux_0073,light_flux_0074,light_flux_0075,light_flux_0076,light_flux_0077,light_flux_0078,light_flux_0079,light_flux_0080,light_flux_0081,light_flux_0082,light_flux_0083,light_flux_0084,light_flux_0085,light_flux_0086,light_flux_0087,light_flux_0088,light_flux_0089,light_flux_0090,light_flux_0091,light_flux_0092,light_flux_0093,light_flux_0094,light_flux_0095,light_flux_0096,light_flux_0097,light_flux_0098,light_flux_0099,light_flux_0100,light_flux_0101
0,10797460.0,Kepler-227 b,K00752.01,1,1.727321,0.728728,1.125654,1.229677,1.327881,1.390140,1.276143,1.281041,1.226772,1.377934,1.024055,1.286729,0.548192,0.725890,0.374606,1.087220,0.728862,0.590892,1.797350,0.849622,0.468157,0.223915,0.740685,0.497284,-0.261216,-0.534282,0.003514,0.138779,-0.117049,-0.027676,-0.442272,0.093172,0.152474,-0.328363,-0.357623,-0.499884,-0.158701,-0.385123,-0.492289,-0.359922,0.323984,-0.130389,-1.000000,0.164044,-0.532362,-0.540776,-0.295388,-0.411368,-0.641894,-0.131383,-0.209675,-0.362780,-0.463537,-0.210418,-0.394556,-0.551926,-0.786976,0.256979,-0.109946,0.019182,-0.286593,-0.041779,-0.096029,-0.142669,-0.169710,-0.240404,-0.291608,-0.260093,0.258748,-0.051029,-0.649683,-0.276882,-0.383267,0.127963,-0.295442,0.069968,0.457335,-0.017516,0.240662,-0.260020,0.629783,0.443599,0.671072,0.611508,0.764126,0.730956,1.178425,1.036392,0.524232,1.298338,1.121857,1.318587,1.390135,0.968716,1.285568,1.216662,0.874477,1.627442,0.960296,0.998560,1.489279
2,10811496.0,NaN,K00753.01,0,1.044298,0.961948,1.000986,0.983043,0.983156,1.037890,1.066556,0.995921,1.004871,1.017373,1.111457,0.952989,0.974510,1.049884,0.963186,0.988939,0.984795,0.966005,1.018833,1.007997,0.974843,1.030709,1.044449,0.982115,0.940630,1.015153,1.000033,1.026868,1.056476,0.964789,1.015093,1.071004,0.992958,0.985321,1.006922,1.044579,1.024812,0.989431,1.029786,1.006301,1.013856,1.091771,0.941154,0.855983,0.742372,0.600087,0.192429,-0.227249,-0.464107,-0.772706,-0.916304,-1.000000,-0.879052,-0.821739,-0.608923,-0.247621,0.196407,0.440931,0.626200,0.798152,0.969366,1.069474,1.024138,0.923610,0.985293,1.041931,0.979435,0.934058,0.987630,1.040991,1.037328,0.917551,1.040966,0.976573,1.033922,0.927375,1.029179,1.028091,1.015323,1.000870,1.001061,1.026146,1.018344,1.019126,0.968186,1.020373,1.065901,0.939004,0.913783,1.065097,1.027615,1.023945,0.999707,0.986407,0.980490,1.018677,0.976698,0.933617,1.067080,1.002989,1.012337
3,10848459.0,NaN,K00754.01,0,-0.289601,-0.370998,-0.441931,-0.410758,-0.379353,-0.507287,-0.488590,-0.507838,-0.520611,-0.440534,-0.585381,-0.571857,-0.598065,-0.602092,-0.689088,-0.608699,-0.641898,-0.750587,-0.653017,-0.675921,-0.717471,-0.769482,-0.760244,-0.756296,-0.768739,-0.680395,-0.719143,-0.679906,-0.801171,-0.740623,-0.813708,-0.802888,-0.763674,-0.784456,-0.660572,-0.769564,-0.700158,-0

In [47]:
for x in range(0, len(modes)):
    pd.DataFrame(modes[x]['df']).to_csv('./unified-lightcurves-' + modes[x]['scope'] + '.csv')

    print("saved", modes[x]['scope'], modes[x]['df'].shape)

saved local (6095, 105)
saved global (6095, 1005)


filtramos los datos del koi para obtener un fichero de datos astronomicos

In [53]:
df_koi
kepids = list(set(modes[0]['df']["kepid"].tolist()))

df_koi_filtered = df_koi[df_koi['kepid'].isin(kepids)].drop_duplicates(subset='kepid', keep="first")
df_koi_filtered.loc[df_koi_filtered["koi_disposition"] != "CONFIRMED", "koi_disposition"] = 0
df_koi_filtered.loc[df_koi_filtered["koi_disposition"] == "CONFIRMED", "koi_disposition"] = 1

In [60]:
df_koi_filtered

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,1,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,615.8,19.5,-19.5,2.26,0.26,-0.15,793.0,NaN,NaN,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,0,CANDIDATE,0.000,0,0,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638.0,NaN,NaN,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,0,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395.0,NaN,NaN,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,1,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406.0,NaN,NaN,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
5,10872983,K00756.01,Kepler-228 d,1,CANDIDATE,1.000,0,0,0,0,11.094321,2.036000e-05,-2.036000e-05,171.201160,0.001410,-0.001410,0.538,0.030,-0.428,4.59450,0.06100,-0.06100,1517.5,24.2,-24.2,3.90,1.27,-0.42,835.0,NaN,NaN,114.81,112.85,-36.70,66.5,1.0,q1_q17_dr25_tce,6046.0,189.0,-232.0,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9558,10031643,K07984.01,NaN,0,FALSE POSITIVE,0.000,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,0.765,0.023,-0.541,4.80600,0.63400,-0.63400,87.7,13.0,-13.0,1.11,0.32,-0.23,929.0,NaN,NaN,176.40,152.77,-77.60,8.4,1.0,q1_q17_dr25_tce,5638.0,169.0,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9559,10090151,K07985.01,NaN,0,FALSE POSITIVE,0.000,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,1.252,0.051,-0.049,3.22210,0.01740,-0.01740,1579.2,4.6,-4.6,29.35,7.70,-2.57,2088.0,NaN,NaN,4500.53,3406.38,-1175.26,453.3,1.0,q1_q17_dr25_tce,5638.0,139.0,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9560,10128825,K07986.01,NaN,0,CANDIDATE,0.497,0,0,0,0,1.739849,1.780000e-05,-1.780000e-05,133.001270,0.007690,-0.007690,0.043,0.423,-0.043,3.11400,0.22900,-0.22900,48.5,5.4,-5.4,0.72,0.24,-0.08,1608.0,NaN,NaN,1585.81,1537.86,-502.22,10.6,1.0,q1_q17_dr25_tce,6119.0,165.0,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9561,10147276,K07987.01,NaN,0,FALSE POSITIVE,0.021,0,0,1,0,0.681402,2.434000e-06,-2.434000e-06,132.181750,0.002850,-0.002850,0.147,0.309,-0.147,0.86500,0.16200,-0.16200,103.6,14.7,-14.7,1.07,0.36,-0.11,2218.0,NaN,NaN,5713.41,5675.74,-1836.94,12.3,1.0,q1_q17_dr25_tce,6173.0,193.0,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [72]:
df_koi_filtered_clean = df_koi_filtered.drop([
    'kepler_name', 'koi_pdisposition','koi_score',
    'koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co',
    'koi_period_err1','koi_period_err2','koi_fpflag_ec',
    'koi_time0bk_err1','koi_time0bk_err2','koi_impact_err1',
    'koi_impact_err2', 'koi_duration_err1', 'koi_duration_err2', 'koi_tce_delivname',
    'koi_depth_err1', 'koi_depth_err2', 'koi_prad_err1', 'koi_prad_err2',
    'koi_teq_err1', 'koi_teq_err2', 'koi_insol_err1', 'koi_insol_err2',
    'koi_steff_err1', 'koi_steff_err2', 'koi_slogg_err1', 'koi_slogg_err2',
    'koi_srad_err1', 'koi_srad_err2', 'koi_tce_plnt_num',
    'kepoi_name'
    # '', '', '', '',
], axis=1)
df_koi_filtered_clean

,kepid,koi_disposition,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,10797460,1,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,35.8,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,10811496,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,76.3,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,10848459,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,505.6,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,10854555,1,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,40.9,6031.0,4.438,1.046,288.75488,48.226200,15.509
5,10872983,1,11.094321,171.201160,0.538,4.59450,1517.5,3.90,835.0,114.81,66.5,6046.0,4.486,0.972,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9558,10031643,0,8.589871,132.016100,0.765,4.80600,87.7,1.11,929.0,176.40,8.4,5638.0,4.296,1.088,298.74921,46.973351,14.478
9559,10090151,0,0.527699,131.705093,1.252,3.22210,1579.2,29.35,2088.0,4500.53,453.3,5638.0,4.529,0.903,297.18875,47.093819,14.082
9560,10128825,0,1.739849,133.001270,0.043,3.11400,48.5,0.72,1608.0,1585.81,10.6,6119.0,4.444,1.031,286.50937,47.163219,14.757
9561,10147276,0,0.681402,132.181750,0.147,0.86500,103.6,1.07,2218.0,5713.41,12.3,6173.0,4.447,1.041,294.16489,47.176281,15.385


pl_trandep = koi_depth <br>
st_tmag = None <br>
st_dist = koi_dor ? <br>
st_teff = koi_steff <br>
st_logg = koi_slogg  <br>
st_rad = koi_srad <br>

In [74]:
pd.DataFrame(df_koi_filtered_clean).to_csv('./unified-stellar-params.csv')